# Solution for Puzzle x

## Imports

In [1]:
import pandas as pd

## Input Data

In [16]:
input = pd.read_csv("input/day_9.csv", sep=",", header=0)
print(f"Number of ranges: {len(input)}")
input.head(5)

Number of ranges: 496


,x,y
0,97603,50471
1,97603,51688
2,97910,51688
3,97910,52889
4,97643,52889


## Part One

In [7]:
areas = []

# Compute area for all pairs of tiles
for i in range(len(input)):
    for j in range(i+1, len(input)):
        x_diff = input.loc[i, "x"] - input.loc[j, "x"] + 1
        y_diff = input.loc[i, "y"] - input.loc[j, "y"] + 1
        areas.append(x_diff*y_diff)

print(f"The largest possible area is {max(areas)}.")

The largest possible area is 4763932976.


## Part Two

In [ ]:
areas = []

# Compute area for all pairs of tiles
for x_0 in range(len(input)):
    for x_1 in range(x_0+1, len(input)):
        for x_2 in range(x_1+1, len(input)):
            for x_2 in range(x_2+1, len(input)):
                x_first = 
                x_diff = input.loc[i, "x"] - input.loc[j, "x"] + 1
                y_diff = input.loc[i, "y"] - input.loc[j, "y"] + 1
                areas.append(x_diff*y_diff)

print(f"The largest possible area is {max(areas)}.")

In [ ]:
import pandas as pd

def is_inside_polygon_ray_casting(point: tuple[int, int], polygon_vertices: list[tuple[int, int]]) -> bool:
    """
    Performs the Point-in-Polygon test using the ray-casting algorithm.
    Returns True if the point is strictly inside the polygon, False otherwise (including boundary).
    
    This works reliably even with large coordinates.
    """
    x, y = point
    n = len(polygon_vertices)
    inside = False
    
    # Iterate over all edges (p1, p2) of the polygon
    p1x, p1y = polygon_vertices[-1]
    
    for i in range(n):
        p2x, p2y = polygon_vertices[i]
        
        # Check if the point lies on an edge (Boundary check - we need strictly inside)
        if (p1x == x and p2x == x and min(p1y, p2y) <= y <= max(p1y, p2y)) or \
           (p1y == y and p2y == y and min(p1x, p2x) <= x <= max(p1x, p2x)):
            return False # Point is on the boundary

        # Ray casting logic: Does the horizontal ray from (x, y) cross the edge (p1, p2)?
        if y > min(p1y, p2y) and y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                # Calculate x-intersection of the horizontal ray (y) with the edge
                x_intersect = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
            
            # If the edge is vertical (p1x == p2x) or the intersection is to the right of the point, the ray crosses the edge.
            if p1x == p2x or x <= x_intersect:
                inside = not inside
                
        p1x, p1y = p2x, p2y
        
    return inside

def check_segment_intersection(x_min, x_max, y_min, y_max, p1, p2):
    """
    Checks if the polygon boundary segment (p1, p2) intersects the INTERIOR
    of the candidate rectangle R_AB defined by [x_min, x_max] x [y_min, y_max].
    """
    x_a, y_a = p1
    x_b, y_b = p2
    
    # The segment is axis-aligned (horizontal or vertical)
    
    if x_a == x_b: # Vertical segment
        # Segment x-coordinate: x_a. Segment y-range: [y_s_min, y_s_max]
        x_s = x_a
        y_s_min, y_s_max = min(y_a, y_b), max(y_a, y_b)
        
        # Check if segment x_s is strictly between rectangle x_min and x_max
        if x_min < x_s < x_max:
            # Check for y-overlap (intersection)
            # Intersection range: [max(y_min, y_s_min), min(y_max, y_s_max)]
            if max(y_min, y_s_min) < min(y_max, y_s_max):
                 # The segment intersects the interior (e.g., if x_s is 5, and the rect is [4,6]x[..])
                return True
                
    elif y_a == y_b: # Horizontal segment
        # Segment y-coordinate: y_a. Segment x-range: [x_s_min, x_s_max]
        y_s = y_a
        x_s_min, x_s_max = min(x_a, x_b), max(x_a, x_b)
        
        # Check if segment y_s is strictly between rectangle y_min and y_max
        if y_min < y_s < y_max:
            # Check for x-overlap (intersection)
            # Intersection range: [max(x_min, x_s_min), min(x_max, x_s_max)]
            if max(x_min, x_s_min) < min(x_max, x_s_max):
                 # The segment intersects the interior
                return True
                
    return False

red_tiles = list(zip(input['x'], input['y']))
n_red = len(red_tiles)
max_area = 0
    
# 1. Iterate through all pairs of red tiles as potential opposite corners
for i in range(n_red):
    for j in range(i + 1, n_red):
        p1 = red_tiles[i]
        p2 = red_tiles[j]
        x1, y1 = p1
        x2, y2 = p2
        
        # Define the bounding box (rectangle)
        x_min, x_max = min(x1, x2), max(x1, x2)
        y_min, y_max = min(y1, y2), max(y1, y2)
        
        # Calculate potential area (width * height, inclusive of endpoints)
        # Area must be calculated on unit tiles: (max - min + 1)
        width = x_max - x_min + 1
        height = y_max - y_min + 1
        current_area = width * height
        
        if current_area <= max_area:
            continue
            
        is_valid_rectangle = True
        
        # 2. Check for Boundary Intersections
        for k in range(n_red):
            p_a = red_tiles[k]
            p_b = red_tiles[(k + 1) % n_red]
            
            if check_segment_intersection(x_min, x_max, y_min, y_max, p_a, p_b):
                is_valid_rectangle = False
                break
        
        if not is_valid_rectangle:
            continue

        # 3. Check for Containment (If no intersection, R_AB is entirely inside or entirely outside)
        test_x, test_y = x_min + 1, y_min + 1
        if width == 1:
            test_x = x_min
        if height == 1:
            test_y = y_min
        
        # Check the center of the first available tile
        test_point = (test_x, test_y) 
        
        center_x = (x_min + x_max) / 2.0
        center_y = (y_min + y_max) / 2.0
        
        if not is_inside_polygon_ray_casting((center_x, center_y), red_tiles):
            is_valid_rectangle = False
            
        if is_valid_rectangle:
            max_area = max(max_area, current_area)

print(f"The largest possible area is {max_area}.")

The largest possible area is 1501292304.
